# IMPORT

In [1]:
import torch
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss, GRUCell
from torch_geometric.data import Data
from sklearn.metrics import average_precision_score

import copy


from torch_geometric.utils import negative_sampling
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.nn import GCNConv, Linear

import torch
import numpy as np

In [2]:
from torch_geometric.datasets import BitcoinOTC

data = BitcoinOTC('bitcoinOTC/')

In [3]:
class ROLANDGNN(torch.nn.Module):
    def __init__(self, input_dim, num_nodes, dropout=0.0, update='moving', loss=BCEWithLogitsLoss):
        
        super(ROLANDGNN, self).__init__()
        #Architecture: 
            #2 MLP layers to preprocess BERT repr, 
            #2 GCN layer to aggregate node embeddings
            #HadamardMLP as link prediction decoder
        
        #You can change the layer dimensions but 
        #if you change the architecture you need to change the forward method too
        #TODO: make the architecture parameterizable
        
        hidden_conv_1 = 64 
        hidden_conv_2 = 32
        self.preprocess1 = Linear(input_dim, 256)
        self.preprocess2 = Linear(256, 128)
        self.conv1 = GCNConv(128, hidden_conv_1)
        self.conv2 = GCNConv(hidden_conv_1, hidden_conv_2)
        self.postprocess1 = Linear(hidden_conv_2, 2)
        
        #Initialize the loss function to BCEWithLogitsLoss
        self.loss_fn = loss()

        self.dropout = dropout
        self.update = update
        if update=='moving':
            self.tau = torch.Tensor([0])
        elif update=='learnable':
            self.tau = torch.nn.Parameter(torch.Tensor([0]))
        elif update=='gru':
            self.gru1 = GRUCell(hidden_conv_1, hidden_conv_1)
            self.gru2 = GRUCell(hidden_conv_2, hidden_conv_2)
        elif update=='mlp':
            self.mlp1 = Linear(hidden_conv_1*2, hidden_conv_1)
            self.mlp2 = Linear(hidden_conv_2*2, hidden_conv_2)
        else:
            assert(update>=0 and update <=1)
            self.tau = torch.Tensor([update])
        self.previous_embeddings = [torch.Tensor([[0 for i in range(hidden_conv_1)] for j in range(num_nodes)]),\
                                    torch.Tensor([[0 for i in range(hidden_conv_2)] for j in range(num_nodes)])]
                                    
        
    def reset_loss(self,loss=BCEWithLogitsLoss):
        self.loss_fn = loss()
        
    def reset_parameters(self):
        self.preprocess1.reset_parameters()
        self.preprocess2.reset_parameters()
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.postprocess1.reset_parameters()

    def forward(self, x, edge_index, edge_label_index, previous_embeddings=None, num_current_edges=None, num_previous_edges=None):        
        #You do not need all the parameters to be different to None in test phase
        #You can just use the saved previous embeddings and tau
        if previous_embeddings is not None: #None if test
            self.previous_embeddings = [previous_embeddings[0].clone(),previous_embeddings[1].clone()]
        if self.update=='moving' and num_current_edges is not None and num_previous_edges is not None: #None if test
            #compute moving average parameter
            self.tau = torch.Tensor([num_previous_edges / (num_previous_edges + num_current_edges)]).clone() # tau -- past weight
        
        current_embeddings = [torch.Tensor([]),torch.Tensor([])]
        
        #Preprocess text
        h = self.preprocess1(x)
        h = F.leaky_relu(h,inplace=True)
        h = F.dropout(h, p=self.dropout,inplace=True)
        h = self.preprocess2(h)
        h = F.leaky_relu(h,inplace=True)
        h = F.dropout(h, p=self.dropout, inplace=True)
        
        #GRAPHCONV
        #GraphConv1
        h = self.conv1(h, edge_index)
        h = F.leaky_relu(h,inplace=True)
        h = F.dropout(h, p=self.dropout,inplace=True)
        #Embedding Update after first layer
        if self.update=='gru':
            h = torch.Tensor(self.gru1(h, self.previous_embeddings[0].clone()).detach().numpy())
        elif self.update=='mlp':
            hin = torch.cat((h,self.previous_embeddings[0].clone()),dim=1)
            h = torch.Tensor(self.mlp1(hin).detach().numpy())
        else:
            h = torch.Tensor((self.tau * self.previous_embeddings[0].clone() + (1-self.tau) * h.clone()).detach().numpy())
       
        current_embeddings[0] = h.clone()
        #GraphConv2
        h = self.conv2(h, edge_index)
        h = F.leaky_relu(h,inplace=True) # if inplace = True, it means it modify h directly
        h = F.dropout(h, p=self.dropout,inplace=True)
        #Embedding Update after second layer
        if self.update=='gru':
            h = torch.Tensor(self.gru2(h, self.previous_embeddings[1].clone()).detach().numpy())
        elif self.update=='mlp':
            hin = torch.cat((h,self.previous_embeddings[1].clone()),dim=1)
            h = torch.Tensor(self.mlp2(hin).detach().numpy())
        else:
            h = torch.Tensor((self.tau * self.previous_embeddings[1].clone() + (1-self.tau) * h.clone()).detach().numpy())
      
        current_embeddings[1] = h.clone()
        #HADAMARD MLP
        h_src = h[edge_label_index[0]]
        h_dst = h[edge_label_index[1]]
        h_hadamard = torch.mul(h_src, h_dst) #hadamard product

        h = self.postprocess1(h_hadamard)
        h = torch.sum(h.clone(), dim=-1).clone()
        
        #return both 
        #i)the predictions for the current snapshot 
        #ii) the embeddings of current snapshot

        return h, current_embeddings
    
    def loss(self, pred, link_label):
        return self.loss_fn(pred, link_label)

In [4]:
from sklearn.metrics import *

def train_single_snapshot(model, data, train_data, val_data, test_data,\
                          last_embeddings, num_current_edges, num_previous_edges,\
                          optimizer, device='cpu', num_epochs=50, verbose=False):
    avgpr_val_max = 0
    best_model = model
    train_data = train_data.to(device)
    best_epoch = -1
    best_current_embeddings = []
    
    tol = 5e-04
    
    for epoch in range(num_epochs):
        model.train()
        ## Note
        ## 1. Zero grad the optimizer
        ## 2. Compute loss and backpropagate
        ## 3. Update the model parameters
        optimizer.zero_grad()
            
        #pred = best_model(train_data)

        pred,\
        current_embeddings =\
            model(train_data.x, train_data.edge_index, train_data.edge_label_index,\
                  last_embeddings, num_current_edges, num_previous_edges)
        
        loss = model.loss(pred, train_data.edge_label.type_as(pred)) #loss to fine tune on current snapshot

        loss.backward(retain_graph=True)  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.

        ##########################################

        log = 'Epoch: {:03d}\n AVGPR Train: {:.4f}, Val: {:.4f}, Test: {:.4f}\n MRR Train: {:.4f}, Val: {:.4f}, Test: {:.4f}\n F1-Score Train: {:.4f}, Val: {:.4f}, Test: {:.4f}\n Loss: {}'
        avgpr_score_val = test(model, val_data, device)
        
        if avgpr_val_max-tol <= avgpr_score_val:
            avgpr_val_max = avgpr_score_val
            best_epoch = epoch
            best_current_embeddings = current_embeddings
            best_model = model
        else:
            break
        
        
    avgpr_score_train = test(model, train_data, device)
    avgpr_score_test = test(model, test_data, device)
    
    return best_model, optimizer, avgpr_score_train, avgpr_score_test, best_current_embeddings

In [5]:
def test(model, test_data, device):
    model.eval()

    test_data = test_data.to(device)

    h, _ = model(test_data.x, test_data.edge_index, test_data.edge_label_index)
    
    pred_cont = torch.sigmoid(h).cpu().detach().numpy()
    
    label = test_data.edge_label.cpu().detach().numpy()
      
    avgpr_score = average_precision_score(label, pred_cont)
    
    return avgpr_score

In [6]:
def train_roland(model, snapshots, hidden_conv1, hidden_conv2, optimizer, device='cpu'):
    num_snap = len(snapshots)
    num_previous_edges = 0
    last_embeddings = [torch.Tensor([[0 for i in range(hidden_conv1)] for j in range(num_nodes)]),\
                                    torch.Tensor([[0 for i in range(hidden_conv2)] for j in range(num_nodes)])]
    avgpr_train_singles = []
    avgpr_test_singles = []
    for i in range(num_snap-1):
        #CREATE TRAIN + VAL + TEST SET FOR THE CURRENT SNAP
        snapshot = copy.deepcopy(snapshots[i])
        snapshot.x = torch.Tensor([[1] for i in range(snapshot.num_nodes)])
        num_current_edges = len(snapshot.edge_index[0])
        transform = RandomLinkSplit(num_val=0.0,num_test=0.25)
        train_data, _, val_data = transform(snapshot)
        if i == 0:
            print("num_current_edges", num_current_edges)
            print("train data is of size", train_data, "val data is of size", val_data)
        test_data = copy.deepcopy(snapshots[i+1])
        test_data.x = torch.Tensor([[1] for i in range(test_data.num_nodes)])
        future_neg_edge_index = negative_sampling(
            edge_index=test_data.edge_index, #positive edges
            num_nodes=test_data.num_nodes, # number of nodes
            num_neg_samples=test_data.edge_index.size(1)) # number of neg_sample equal to number of pos_edges
        #edge index ok, edge_label concat, edge_label_index concat
        num_pos_edge = test_data.edge_index.size(1)
        test_data.edge_label = torch.Tensor(np.array([1 for i in range(num_pos_edge)] + [0 for i in range(num_pos_edge)]))
        test_data.edge_label_index = torch.cat([test_data.edge_index, future_neg_edge_index], dim=-1)
        
        #TRAIN AND TEST THE MODEL FOR THE CURRENT SNAP
        model, optimizer, avgpr_train, avgpr_test, last_embeddings =\
            train_single_snapshot(model, snapshot, train_data, val_data, test_data,\
                                  last_embeddings, num_current_edges, num_previous_edges,\
                                  optimizer)
        
        #SAVE AND DISPLAY EVALUATION
        print(f'Snapshot: {i}\n AVGPR Train: {avgpr_train}, Test: {avgpr_test}\n')
        avgpr_train_singles.append(avgpr_train)
        avgpr_test_singles.append(avgpr_test)
        
        #COMPUTE NEW NUMBER OF EDGES
        num_previous_edges = num_previous_edges + num_current_edges
        
    avgpr_train_all = sum(avgpr_train_singles)/len(avgpr_train_singles)
    avgpr_test_all = sum(avgpr_test_singles)/len(avgpr_test_singles)
    
    print(f'AVGPR over time: Train {avgpr_train_all}, Test: {avgpr_test_all}')

In [7]:
torch.autograd.set_detect_anomaly(True)

## Train

In [8]:
input_dim = data[0].num_node_features
hidden_conv1 = 64
hidden_conv2 = 32
num_classes = 2
num_nodes = data[0].num_nodes

In [9]:
model = ROLANDGNN(input_dim, num_nodes, update='gru')
model.reset_parameters()

In [10]:
weight_decay = 5e-3
#optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=weight_decay)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01, weight_decay = weight_decay)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1, weight_decay = weight_decay)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, weight_decay=weight_decay)

In [11]:
train_roland(model, data, hidden_conv1, hidden_conv2, optimizer)

num_current_edges 41
train data is of size Data(edge_index=[2, 31], edge_attr=[31], num_nodes=6005, x=[6005, 1], edge_label=[62], edge_label_index=[2, 62]) val data is of size Data(edge_index=[2, 31], edge_attr=[31], num_nodes=6005, x=[6005, 1], edge_label=[20], edge_label_index=[2, 20])
Snapshot: 0
 AVGPR Train: 0.8083954266947113, Test: 0.7661058619796044

Snapshot: 1
 AVGPR Train: 0.9624371745602568, Test: 0.8159440285762837



KeyboardInterrupt: 